In [ ]:
pip install transformers datasets sentencepiece flask


In [ ]:
from google.colab import files
uploaded = files.upload()  # Opens a file browser to upload


In [ ]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Example
inputs = tokenizer("summarize: " + plot_text, return_tensors="pt", max_length=512, truncation=True)


In [ ]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

model = T5ForConditionalGeneration.from_pretrained("t5-small")

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # Use your cleaned dataset here
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

trainer.train()


In [ ]:
model.save_pretrained("movie-summary-model")
tokenizer.save_pretrained("movie-summary-model")


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained("movie-summary-model")
tokenizer = T5Tokenizer.from_pretrained("movie-summary-model")

def generate_summary(plot):
    inputs = tokenizer("summarize: " + plot, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(inputs["input_ids"], max_length=100, min_length=30, length_penalty=2.0, num_beams=4)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)
